In [1]:
from config import hf_cache_dir

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model = AutoModelForCausalLM.from_pretrained(
    "tomg-group-umd/huginn-0125", torch_dtype=torch.bfloat16, trust_remote_code=True,
    cache_dir=hf_cache_dir)
tokenizer = AutoTokenizer.from_pretrained("tomg-group-umd/huginn-0125")


A new version of the following files was downloaded from https://huggingface.co/tomg-group-umd/huginn-0125:
- raven_config_minimal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


raven_modeling_minimal.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tomg-group-umd/huginn-0125:
- raven_modeling_minimal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
ctrl_model = AutoModelForCausalLM.from_pretrained(
    "tomg-group-umd/step-00010720-baseline_2_0", torch_dtype=torch.bfloat16, trust_remote_code=True,
    cache_dir=hf_cache_dir)
ctrl_tokenizer = AutoTokenizer.from_pretrained("tomg-group-umd/step-00010720-baseline_2_0")
ctrl_model.config.test_time_noise = 0
ctrl_model.config.test_time_noise_type = "fixed"

config.json: 0.00B [00:00, ?B/s]

raven_config_minimal.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tomg-group-umd/step-00010720-baseline_2_0:
- raven_config_minimal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


raven_modeling_minimal.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tomg-group-umd/step-00010720-baseline_2_0:
- raven_modeling_minimal.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [3]:
device = 'cuda:0'

In [ ]:
model = model.to(device)
ctrl_model = ctrl_model.to(device)

# Prompt completion with GenerationConfig

In [ ]:
prompt = "The capital of the country where the Eiffel tower is located is "  # ctrl can do this, r=1 can't
prompt = "The capital of the country where the Big Ben is located is "  # ctrl and r=1 can't do this
prompt = "The capital of the country where the Statue of Liberty is located is "  # no one can do this
prompt = "The capital of the country where the Grand Canyon is located is "  # no one can do this
prompt = "Buckingham Palace is located in the country where the capital is "  # ctrl and r=1 can't do this
prompt = "The Louvre is located in the country where the capital is "  # ctrl can do this, r=1 can't
prompt = "The Great Sphinx of Giza is located in the country where the capital is "  # ctrl and r=1 can't do this
prompt = "Osaka is located in the country where the capital is " # ctrl and r=1 can't do this
prompt = "San Francisco is located in the country where the capital is "  # no one can do this
prompt = "Munich is located in the country where the capital is " # ctrl and r=1 can't do this

In [128]:
model.eval()
config = GenerationConfig(max_length=50, stop_strings=["<|end_text|>", "<|end_turn|>"], 
                          use_cache=True,
                          do_sample=False, temperature=None, top_k=None, top_p=None, min_p=None, 
                          return_dict_in_generate=True,
                          eos_token_id=65505,bos_token_id=65504,pad_token_id=65509)


input_ids = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to(device)
outputs = model.generate(input_ids, config, tokenizer=tokenizer, num_steps=32)
print(tokenizer.decode(outputs['sequences'].squeeze()))

<|begin_text|>Munich is located in the country where the capital is ​Berlin. The city is located in the south of the country, in the south-east of the state of Bavaria. The city is located in the south-east


In [129]:
# No recurrent steps
outputs = model.generate(input_ids, config, tokenizer=tokenizer, num_steps=2)
print(tokenizer.decode(outputs['sequences'].squeeze()))

<|begin_text|>Munich is located in the country where the capital is ichthia. Munich is a city in the middle of Munich in the country. Munich is the capital of Munich in the country. Munich is the capital of


In [130]:
# Baseline model
ctrl_model.eval()
config = GenerationConfig(max_length=50, stop_strings=["<|end_text|>", "<|end_turn|>"], 
                          use_cache=True,
                          do_sample=False, temperature=None, top_k=None, top_p=None, min_p=None, 
                          return_dict_in_generate=True,
                          eos_token_id=65505,bos_token_id=65504,pad_token_id=65509)

input_ids = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to(device)
outputs = ctrl_model.generate(input_ids, config, tokenizer=tokenizer, num_steps=1)
print(tokenizer.decode(outputs['sequences'].squeeze()))

<|begin_text|>Munich is located in the country where the capital is ​the city of Munich. It is the largest city in the country and the second largest city in the world. Munich is the largest city in the country and the second largest


# Chat template

In [30]:
messages = []
#messages.append({"role": "system", "content" : "You are a helpful assistant."})
messages.append({"role": "user", "content" : "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"})
chat_input = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(chat_input)
input_ids = tokenizer.encode(chat_input, return_tensors="pt", add_special_tokens=False).to(device)

output_ids = model.generate(input_ids, config, num_steps=40, tokenizer=tokenizer)


<|begin_text|><|begin_header|>user<|end_header|>

Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|end_turn|><|begin_header|>Huginn<|end_header|>




In [31]:
print(tokenizer.decode(output_ids['sequences'].squeeze()))

<|begin_text|><|begin_header|>user<|end_header|>

Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|end_turn|><|begin_header|>Huginn<|end_header|>

In April, Natalia sold 48 clips.
In May, she sold half as many clips as in April, so she sold 48 / 2 = 24 clips.
To find the total number of clips Natalia sold in April and May, we add the number of clips she sold in each month: 48 + 24 = 72.
So Natalia sold \boxed{72} clips altogether in April and May.<|end_turn|>
